# Cuts Optimization's correlations

Over the last years, **Machine Learning** tools have been successfully applied to problems in high-energy physics. For example, for the classification of physics objects. Supervised machine learning algorithms allow for significant improvements in classification problems by taking into account observable correlations and by learning the optimal selection from examples, e.g. from Monte Carlo simulations.

# Installing Libraries
There are certain libraries that we use but do not come in with the normal version of python, therefore, they are needed to be installed.

In [ ]:
%%capture
!pip install hipe4ml
!pip install uproot
!pip install xxhash
!pip install lz4
!pip install plotly
!pip install plotly --upgrade

# Importing the Libraries

**Numpy** is a powerful library that makes working with python more efficient, so we will import it and use it as np in the code. **Pandas** is another useful library that is built on numpy and has two great objects *series* and *dataframework*. Pandas works great for *data ingestion* and also has *data visualization* features. **Matplotlib** and **Seaborn** come handy in plotting and visualizing the data. From **Hipe4ml** we import **TreeHandler** and with the help of this function we will import our *Analysis Tree* to our notebook. We will also need some functions of **Scipy** for fittintg.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from hipe4ml.tree_handler import TreeHandler
#from hipe4ml import plot_utils
from matplotlib.patches import ConnectionPatch
from scipy.stats import binned_statistic as b_s
from scipy.stats import linregress
from sklearn import datasets, linear_model
import statsmodels.api as sm

# Importing the data
CBM has a modified version of the cern's root software and it contains the simulated setup of CBM. Normally, a model generated input file, for example a URQMD 12 AGeV, is passed through different macros. These macros represent the CBM setup and it is like taking particles and passing them through a detector. These particles are registered as hits in the setup. Then particles' tracks are reconstructed from these hits using cellular automaton and Kalman Filter mathematics.


CBM uses the **TTree** format of cern root to store information. To reduce the size of these root files a modified tree file was created by the name of Analysis tree. This Analysis tree file contains most of the information that we need for physics analysis. 

A lambda baryon mostly decays into a proton and a pion. In this example, we download two files that were converted to a plain TTree format (simplest structure to be read by Python). The first one contains mostly background candidates for lambda i.e. protons and pions tracks which do not come from a lambda decay. The second file contains mostly signal candidates of lamba i.e. it contains protons and pions which come from a lambda decay.

In [ ]:
%%capture
!curl -L https://cernbox.cern.ch/index.php/s/AnY17jXGwSpp6QQ/download --output bg.root
!curl -L https://cernbox.cern.ch/index.php/s/RHOSK5CluLf9yso/download --output signal.root

The following lines of code converts the root files into a pandas dataframe objects. With the help of a selection cut, we select only signal candidates and background candidates from their respective data sets. Another selection cut chooses lambda candidates only in the $\pm 5\sigma$ region around the mean of the lambda mass peak. Similarly, we select the background candidates outside this $\pm 5\sigma$ region.

In [4]:
# Variables for 5sigma region selection
left5sigma_signal = '1.108'
right5sigma_signal = '1.1227'
left5sigma_bg = '1.1077'
right5sigma_bg = '2.00'
# We import the root files into our jupyter notebook
input_tree_bg = TreeHandler('bg.root','PlainTree')
input_tree_signal = TreeHandler('signal.root','PlainTree')

# We only select lambda candidates
true_signal = input_tree_signal.get_subset("LambdaCandidates_is_signal == 1 & LambdaCandidates_mass>'@left5sigma' & LambdaCandidates_mass<'@right5sigma'")
# Then we store the information in a dataframe object, sgnal, of Pandas library
sgnal = true_signal.get_data_frame()

# Similarly for the background
true_background = input_tree_bg.get_subset("LambdaCandidates_is_signal == 0 & '@left5sigma_bg'< LambdaCandidates_mass  < '@right5sigma_bg'")
bg=true_background.get_data_frame()

ValueError: could not convert string to float: '@left5sigma'

In [ ]:
#The labels of the columns in the df data frame are having the prefix LambdaCandidates_ so we rename them
new_labels= ['chi2geo', 'chi2primneg', 'chi2primpos', 'chi2topo', 'cosineneg',
       'cosinepos', 'cosinetopo', 'distance', 'eta', 'l', 'ldl',
       'mass', 'p', 'pT', 'phi', 'px', 'py', 'pz', 'rapidity',
             'x', 'y', 'z', 'daughter1id', 'daughter2id', 'isfrompv', 'pid', 'issignal']

sgnal.columns = new_labels
bg.columns = new_labels

#Let's see how the dataframe object df looks like
sgnal

The above data frame object has some columns/features and for them at the very last column (issignal) the true Monte Carlos information is available. This MC information tells us whether this reconstructed particle was originally produced as a decaying particle or not. So a value of 1 means that it is a true candidate and 0 means that it is not.

# Data Cleaning
Sometimes a data set contains entries which are outliers or does not make sense. For example, infinite values or NaN entries. We clean the data by removing these entries. 

Similarly, CBM is a fixed target experiment so there are certain conditions which the data has to satisfy before it is considered as reliable data.So we apply certain limits on the data sets.

Ofcourse, we lose some data points but these outliers sometimes cause problems when we perform analysis. 

In [ ]:
def clean_df(df):
    # let's treat all the infinite, inf, values by nan and then we drop all the null entries
    with pd.option_context('mode.use_inf_as_na', True):
        df = df.dropna()
    #Experimental constraints
    is_good_mom = (df['pz'] > 0) & (df['p']<20) & (df['pT']<3)
    is_good_coord = (abs(df['x']) < 100) & (abs(df['y']) < 100) & (df['z']>0) & (df['z']<80)
    is_good_params = (df['distance'] > 0) & (df['distance'] < 100) & (df['chi2geo']>0) & (df['chi2geo'] < 1000) & (df['cosinepos'] > 0.5) & (df['chi2topo'] > 0) & (df['chi2topo'] < 100000) & (df['cosineneg']>0.1) & (df['eta']>1) & (df['eta']<6.5) & (df['l']<80) & (df['ldl']>0) & (df['ldl']<5000)
    is_good_daughters = (df['chi2primneg']>0) & (df['chi2primneg'] < 3e7) & (df['chi2primpos']>0) & (df['chi2primpos']<1e6)
    is_good_mass = (df['mass']>0) & (df['mass']<2.5)

    is_good_df = (is_good_mom) & (is_good_coord) & (is_good_params) & (is_good_daughters) & (is_good_mass)

    return df[is_good_df]

In [ ]:
background = clean_df(bg)
signal = clean_df(sgnal)

# Correlation
We find the correlation of all variables with signal and background candidates. We use the pearson correlation coefficient (linear correlation) for our analysis. It is defined as 
$$
\rho = \frac{COV(X,Y)}{\sigma_X \times \sigma_Y}
$$
Here, COV(X,Y) is the covariance of the variable X and Y, and $\sigma_X$ and $\sigma_Y$ are the standard deviations of the variables. Pearson co-efficient is useful for linear correlation but it fails to take into account outliers and non-linear correlation. $\rho \> 0$ means postive while the opposite means negative correlation between two variables. 

This correlation function comes in built in the pandas library so we are using it. This function can also find other non-linear correlation coefficients like kendall and spearman. 

In [ ]:
variables_to_draw = ['chi2geo', 'chi2primneg', 'chi2primpos', 'chi2topo', 'cosineneg',
       'cosinepos', 'cosinetopo', 'distance', 'eta', 'l', 'ldl',
       'mass', 'p', 'pT', 'phi', 'px', 'py', 'pz', 'rapidity']

In [ ]:
def correlation_graph(df, variables):
    # The variables pid, isfrompv and issignal are not that much varying so we remove them
    new_df = df[variables]
    # Using the pandas correlation function corr we find the correlation
    df_correlation_all = new_df.corr(method='pearson')
    
    #The cosmetics of the graph
    fig, ax = plt.subplots(figsize=(20,15))  #figure size
    cmap = sn.diverging_palette(240, 10, as_cmap=True, n=200) #color map
    cax = sn.heatmap(df_correlation_all, annot=True,cbar_kws={"shrink": .5},  cmap=cmap,  vmin=-1, vmax=1, 
                 center=0)
    ax.set_xticks(np.arange(0, len(df_correlation_all.columns), step=1))
    ax.set_xticklabels(df_correlation_all.columns, fontsize=15, rotation =70)
    ax.set_yticklabels(df_correlation_all.columns, fontsize=15)
    ax.set_title('Correlations of all variables', fontsize = 20)
    #fig.savefig("correlations.png")

In [ ]:
# Insert the data frame object in the following brackets and it will generate the correlation graph for it. Right now use either signal or background
correlation_graph(background, variables_to_draw)

The correlation graph of the background variables shows that cosinepos is correlated with mass. To check whether it is a real correlation or a statistical fluctuation we make our own correlation function. 

## Correlations by formula
The following function calculates the correlation along with the standard error of the mean (SEM) of the input variable with all the other variables. The standard error of the mean is defined as $ SEM = \frac{\sigma}{\sqrt{n}}$. Here $\sigma$ is the standard deviation of a variable. It will put error bars on each bin.

The function accepts 3 variables, a data frame object in the first input, a list of strings to be correlated with the third input (a string).

In [ ]:
def calculate_correlation(df, vars_to_corr, target_var) :
    
    from scipy.stats import sem

    mean = df[target_var].mean()
    sigma = df[target_var].std()

    correlation = []
    error = []
    
    for j in vars_to_corr : 
        mean_j = df[j].mean()
        sigma_j = df[j].std()
        
        cov = (df[j] - mean_j) * (df[target_var] - mean) / (sigma*sigma_j)        
        correlation.append(cov.mean())
        error.append(sem(cov))
    
    return correlation, error

In [ ]:
# Provide the data frame object first, then also inside the brackets of list and then write the variable inside inverted commas ''.
# For signal
corr_signal, corr_signal_errors = calculate_correlation(signal, list(signal), 'mass')
# For background
corr_bg, corr_bg_errors = calculate_correlation(background, list(background), 'mass')

In [ ]:
# Plotting the correlations of various variables with mass along with the errors
fig, ax = plt.subplots(figsize=(20,10))
plt.errorbar(list(signal), corr_signal, yerr=corr_signal_errors, fmt='')
plt.errorbar(list(background), corr_bg, yerr=corr_bg_errors, fmt='')
ax.grid(zorder=0)
ax.set_xticklabels(signal.columns, fontsize=15, rotation =90)
plt.legend(('signal','background'), fontsize = 15)
fig.tight_layout()
plt.title('Correlation of all variables with mass along with SEM', fontsize = 15)
#fig.savefig("hists.png")

2d histograms

## Scatter plot between variables
To analyze the correlation between the mass variable and the cosine of the angle between the proton and the lambda for the background set, we multi-differential analysis.

We make a function which takes in a data frame object in the first input, and then two variables from this df in the next inputs. This function takes the entries of the variables and distributes them in 100 bins. The function then plots the bin centers of the first variable on the x-axis and the mean values of the bins of the second variable on the y-axis, along with its bin stds.

In [ ]:
#for variable bin size, use the following bins
non_uniform_binning = [1.07032418, 1.07962069, 1.08891719, 1.0982137 , 1.1075102 ,
       1.11680671, 1.12610322, 1.13539972, 1.14469623, 1.15399273,
       1.16328924, 1.17258574, 1.18188225, 1.19,1.20977176, 1.25625429,
       1.30273682, 1.34921935, 1.39570187, 1.4421844 , 1.48866693,
       1.53514946, 1.58163198, 1.62811451, 1.67459704, 1.72107956,
       1.76756209, 1.81404462, 1.86052715, 1.90700967, 1.9534922 ,
       1.99997473]
bb = ['1.07', '', '', '', '1.1', '', '', '1.13', '', '', '', '', '', '', '1.2', '1.25', '1.3', '1.34', '1.39', '1.44', '1.48', '1.53', '1.58', '1.62', '1.67', '1.72', '1.76', '1.814', '1.86', '1.9', '1.953', '1.99']

In [ ]:
def profile_plot(df,variable_xaxis,variable_yaxis, binning):
    fig, axs = plt.subplots(figsize=(20, 15))
    # Distributing the data into 100 bins
    bin_means, bin_edges, binnumber = b_s(df[variable_xaxis],df[variable_yaxis], statistic='mean', bins=binning)
    bin_std, bin_edges, binnumber = b_s(df[variable_xaxis],df[variable_yaxis], statistic='std', bins=binning)
    bin_count, bin_edges, binnumber = b_s(df[variable_xaxis],df[variable_yaxis], statistic='count', bins=binning)
    bin_width = (bin_edges[1] - bin_edges[0])
    bin_centers = bin_edges[1:] - bin_width/2
    plt.errorbar(x=bin_centers, y=bin_means, yerr=(bin_std/np.sqrt(bin_count)), linestyle='none', marker='.',mfc='red', ms=10)
    # Fitting a line on the data  
    X = bin_centers
    y = bin_means
    X2 = sm.add_constant(X) 
    est = sm.OLS(y, X2)
    est2 = est.fit()
    p_value1=str(est2.pvalues[0])
    p_value2=str(est2.pvalues[1])
    slope, intercept, r_value, p_value, std_err = linregress(x=bin_centers, y=bin_means)
    print("summary()\n",est2.summary())
    print("pvalues\n",est2.pvalues)
    print("tvalues\n",est2.tvalues)
    print("rsquared\n",est2.rsquared)
    print("rsquared_adj\n",est2.rsquared_adj)
    print('sum of squared residuals = ', np.sum(est2.resid))
    
    from sklearn.metrics import mean_squared_error
    print('mean squared error: ',mean_squared_error(bin_means, intercept + slope*bin_centers))

    predictions = est2.predict(X2)

    print(est2.predict(X2[:3,:]))

    plt.vlines(x=1.115,ymin=0.96,ymax=1.01, color='r', linestyle='-')
  
    #plotting
    plt.plot(bin_centers, intercept + slope*bin_centers, 'b', label='fitted line'+
             " with $R^2$-squared: %f" % r_value**2+'\n and the p values are \n ['+p_value1+'  '+ p_value2+'] \n $\chi^2$')
    plt.legend(fontsize=15)
    plt.title('Mean of ' +variable_yaxis+ ' plotted versus bin centers of '+variable_xaxis, fontsize=18)
    plt.xlabel("Bin centers", fontsize=18)
    plt.ylabel("Mean of each bin with the SEM ($\dfrac{bin\ std}{\sqrt{bin\ count}}$) of bin", fontsize=18)
    #for non-uniform binning labels
    axs.set_xticks(non_uniform_binning)
    axs.set_xticklabels(bb)
    axs.tick_params(labelsize=18)
    fig.tight_layout()
    fig.savefig("hists.png")

In [ ]:
profile_plot(background,'mass','cosinepos',non_uniform_binning)

In [ ]:
def two_D_hist(var_xaxis, var_yaxis):
    import matplotlib as mpl
    fig, axs = plt.subplots(figsize=(15, 10))
    plt.hist2d(var_xaxis,var_yaxis, bins=100, norm=mpl.colors.LogNorm())
    plt.xlabel(var_xaxis.name, fontsize=15)
    plt.ylabel(var_yaxis.name, fontsize=15)
    plt.title("2D histogram having "+var_xaxis.name +" on the x-axis and "+var_yaxis.name+" on the y-axis", fontsize=15)
    axs.tick_params(labelsize=18)
    fig.tight_layout()
    plt.show()

In [ ]:
two_D_hist(background['mass'], background['cosinepos'])

#3D plot
This one is generated for just fun, try to play around with three variables at a time

In [ ]:

import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.

# Configure the trace.
trace = go.Scatter3d(
    x=background['mass'],  # <-- Put your data instead
    y=background['pT'],  # <-- Put your data instead
    z=background['cosinepos'],  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 1,
        'opacity': 0.8,
    },
    
)


# Configure the layout.
layout = go.Layout(scene = dict(xaxis_title='mass',yaxis_title='PT', zaxis_title='cosinepos',
        xaxis = dict(nticks=6, range=[1.09,2]),
                     yaxis = dict(nticks=6, range=[0,3],),
                     zaxis = dict(nticks=6, range=[0.5,1],),),
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)



data = [trace]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)
